# Cleaning and Exploring the Demographics and Programs data

In [1]:
import pandas as pd
import openpyxl

In [2]:
df = pd.read_excel('../data/All demographics and programs.xlsx')

In [ ]:

def data_cleaning(demographics_clean):
    """
    This function drops the first name and last name columns since they don't have any names, 
    drops Single Parent column because only 1 row has 'No' and rest are null,
    drops Ethnicity Hispanic/Latino column because it is all null,
    drops Ex-Offender because only 1 row has No and the rest are null,
    combines values for Transgender M to F and F to M to the category 'Transgender' because there were only 16 that identifed M to F or F to M
    Splits the 'Race' column into 4 because it has 1st, 2nd, 3rd, and 4th levels of selection separated by ;,
    finally duplicate rows were dropped which also, conveniently, eliminated and duplicates for Auto ID
    """
    demographics_clean = df.drop(columns=['First Name', 'Last Name', 'Ethnicity Hispanic/Latino',
                                 'Single Parent', 'Ex-Offender', 'Program: Program Name', 'Outcome'])
    demographics_clean['Gender'].str.replace(
        'Transgender female to male', 'Transgender')
    demographics_clean['Gender'].str.replace(
        'Transgender male to female', 'Transgender')
    splitting = demographics_clean['Race'].str.split(';', expand=True)
    splitting.columns = [f'Race_{i+1}' for i in range(splitting.shape[1])]
    demographics_clean = pd.concat([demographics_clean, splitting], axis=1)
    demographics_clean = demographics_clean.drop(columns=['Race'])
    demographics_clean = demographics_clean.drop_duplicates()
    return (demographics_clean)


demographics_clean = data_cleaning(df)

In [10]:
demographics_clean.head()

,Auto Id,Gender,Veteran,Justice Involved,Race_1,Race_2,Race_3,Race_4
0,202107-1206,Male,No,NaN,Black or African American,None,None,None
3,202108-5167,Male,No,No,Asian,None,None,None
4,202108-5171,Male,NaN,NaN,Black or African American,None,None,None
5,202108-5172,Female,NaN,NaN,White,None,None,None
6,202107-1208,Female,NaN,NaN,Black or African American,None,None,None


In [ ]:
# 2 remaining questions:
# what does 'not applicable' mean for 'Veteran' column
# what does 'Justice Involved' column mean?

In [7]:
def remove_unused_columns(df: pd.DataFrame) -> pd.DataFrame:
    """
    Remove columns with mostly null values or unnecessary information.

    Args:
        df (pd.DataFrame): Input dataframe containing demographic data.

    Returns:
        pd.DataFrame: Dataframe with specified columns removed.
    """
    columns_to_drop = [
        'First Name', 'Last Name', 'Ethnicity Hispanic/Latino',
        'Single Parent', 'Ex-Offender', 'Program: Program Name', 'Outcome'
    ]
    return df.drop(columns=columns_to_drop, errors='ignore')


def normalize_gender(df: pd.DataFrame) -> pd.DataFrame:
    """
    Normalize gender values by combining 'Transgender male to female'
    and 'Transgender female to male' into a single 'Transgender' category.

    Args:
        df (pd.DataFrame): Input dataframe containing a 'Gender' column.

    Returns:
        pd.DataFrame: Dataframe with normalized gender values.
    """
    df['Gender'] = df['Gender'].replace({
        'Transgender male to female': 'Transgender',
        'Transgender female to male': 'Transgender'
    })
    return df


def split_race_column(df: pd.DataFrame) -> pd.DataFrame:
    """
    Split the 'Race' column into multiple columns if multiple races are selected.

    Args:
        df (pd.DataFrame): Input dataframe containing a 'Race' column.

    Returns:
        pd.DataFrame: Dataframe with new columns Race_1, Race_2, etc.
    """
    splitting = df['Race'].str.split(';', expand=True)
    splitting.columns = [f'Race_{i+1}' for i in range(splitting.shape[1])]
    df = pd.concat([df.drop(columns=['Race']), splitting], axis=1)
    return df


def drop_duplicates(df: pd.DataFrame) -> pd.DataFrame:
    """
    Remove duplicate rows from the dataframe.

    Args:
        df (pd.DataFrame): Input dataframe.

    Returns:
        pd.DataFrame: Dataframe without duplicate rows.
    """
    return df.drop_duplicates()


def data_cleaning(demographics_clean: pd.DataFrame) -> pd.DataFrame:
    """
    Perform full data cleaning process on demographics data.

    Steps include:
    - Removing unused or mostly null columns
    - Normalizing gender values
    - Splitting the 'Race' column into multiple race columns
    - Dropping duplicate rows

    Args:
        demographics_clean (pd.DataFrame): Raw demographics dataframe.

    Returns:
        pd.DataFrame: Cleaned dataframe ready for analysis.
    """
    demographics_clean = remove_unused_columns(demographics_clean)
    demographics_clean = normalize_gender(demographics_clean)
    demographics_clean = split_race_column(demographics_clean)
    demographics_clean = drop_duplicates(demographics_clean)
    return demographics_clean

In [8]:
df = data_cleaning(df)
df.head()

,Auto Id,Gender,Veteran,Justice Involved,Race_1,Race_2,Race_3,Race_4
0,202107-1206,Male,No,NaN,Black or African American,None,None,None
3,202108-5167,Male,No,No,Asian,None,None,None
4,202108-5171,Male,NaN,NaN,Black or African American,None,None,None
5,202108-5172,Female,NaN,NaN,White,None,None,None
6,202107-1208,Female,NaN,NaN,Black or African American,None,None,None


In [3]:
from cleaning import DemographicsCleaning

In [4]:
demographics_clean = DemographicsCleaning.clean(df)
demographics_clean.head()

,Auto Id,Gender,Veteran,Justice Involved,Race_1,Race_2,Race_3,Race_4
0,202107-1206,Male,No,NaN,Black or African American,None,None,None
3,202108-5167,Male,No,No,Asian,None,None,None
4,202108-5171,Male,NaN,NaN,Black or African American,None,None,None
5,202108-5172,Female,NaN,NaN,White,None,None,None
6,202107-1208,Female,NaN,NaN,Black or African American,None,None,None
